# Getting Started with Error Mitigation with mitiq on Braket 

A key characteristic of realistic quantum systems is the rate at which they couple to external environment. Thus, unitary evolutions couple with bath environment resulting in information loss or distortion in the system of interest. In digital quantum computing, we can understand this as errors affecting our simulation. Thus the most trustworthy approach is to correct these errors, which in principle can be done using so called quantum error correction. 

Fully quantum error corrected schemes are not here however, and so we can instead try to mitigate errors or harness the noisy signal within a noisy quantum system, up to the decay time of our system. Quantum error mitigation (QEM) techniques attempts to do just this, similar to improving the signal-to-noise ratio of a noisy signal, generally at the cost of repeated measurements. 

In this notebook we will look at `mitiq`, a python library for performing quantum error mitigation. 

### A definition of error mitigation? 

Quantum error mitigation, or just error mitigation, can be defined as tools or techniques which can reduce or `mitigate` the effect of quantum errors in a system, using approaches without active feedback. That is, extra copies of states can be used, and post processing channels applied to a system, in order to improve the quality of a result, but we do not allow for active feedback. This has helped to theoretically define the limits of quantum error mitigation. 

#### The cost of Error Mitigation

Error mitigation techniques have been shown to have a fundamentally exponential scaling for unbiased results. That is, to correct all error for arbitrary input states requires an exponential amount of resources. This exponential requirement is present in the variance of the estimation, which naively scales as $e^{\lambda \times IF \times N}$, where $N$ is the number of relevant gates, $IF$ is some limiting gate infidelity, and $\lambda$ relates to the method. Probabilistic error cancellation for instance, is lower bounded by $\lambda = 4$, while zero-noise extrapolation can reach $\lambda = 2$, and post-selection, $\lambda = 1$.

We include multiple references at the end for the interested reader. 


### Using mitiq and Braket

mitiq is an open source Python toolkit for implementing error mitigation techniques, which covers a breadth of different methodoloies and tools. 

In these notebooks, we show to utilize mitiq with Amazon Braket, specifically focusing on utilizing Program Sets to orchestrate our job submissions. 

However, it is critical to know what is worth spending more time and executions on. Utilizing ProgramSets is only half the battle - by batching we can generally reduce costs, seeing up to 100x decreases in cost, but we also want to be aware of where our variances are coming from, and that we have allocated our resources accordingly. This is a challenging task, and generally can vary depending on problems, circuits, devices, and mitigation techniques.



The mitiq library provides numerous error mitigation approaches as pre-implemented strategies. These include (but are not limited to):

- Readout error mitigation (`mitiq.rem`)
- Zero noise extrapolation (`mitiq.zne`)
- Probabilistc Error Cancellation (`mitiq.pec`)
- Probabilistic Error Amplification (`mitiq.pea`)
- Pauli Twirling (`mitiq.pt`)
- Robust Shadow Estimation (`mitiq.rse`)
- Quantum Subspace Expansions for Stabilizers (`mitiq.qse`)
- Clifford Data Regression (`mitiq.cdr`)
- Virtual Distillation (`mitiq.vd`)
- Digital Dynamical Decopuling (`mitiq.ddd`)

and more. 


## Overview on mitiq 
 
mitiq contains two methods for applying error mitigation generally. The first can be thought of as a simple function call which passes a Circuit and scalars. 

```
from mitiq.zne import execute_with_zne
zne_result = execute_with_zne(circuit, executor, *args)
```
The second unpacks this procedure, separating the creation of circuits and their application and reassembly:
```
from mitiq.zne import construct_circuits, combine_results
modified_circuits = construct_circuits(circuit)
raw_data = executor(modified_circuits)
zne_result = combine_results(raw_data)
```

in the `mitiq_braket_tools.py` file, we develop a `braket_mitiq_executor` function which can easily be used with the first pattern, and meets the requirements of a `mitiq.executor.Executor` object. The second we will explore more in a future notebook, and is develop in the parent `tools` folder, i.e. in the `tools/program_set_tools.py` file. 

Further details can be found at https://mitiq.readthedocs.io/en/stable/guide/guide.html

### Using Verbatim Circuits

For most of these approaches, we will want to treat the circuit using the Verbatim box. This creates a more reliable thread between the submitted circuits and the hypothetical noise model which is being manipulated, and allows us to skip the Braket service compiler, i.e. run the circuit in a verbatim manner.  

That is, given a circuit of interest, we can 

1. Pass our circuit through a compiler / transpiler 
2. Pass our circuit through mitiq to create copies or multiple instances
3. Run our circuits on the Braket service using Verbatim compilation 

For simplicity, we will use the qiskit-braket-provider to harness the Qiskit transpiler, though alternatively the Braket compiler can be used as well by submitting a job to the service against a compiler and returning the output circuit. 


### Installing mitiq

mitiq is distributed under the GPL license, and so is not included by default with the amazon-braket-examples or in the Amazon Braket notebook instances. To install, uncomment the first line in the next code block, and restart the notebook. Mitiq utilizes Cirq as the backend, and often times will represent circuits using their representation. 

In [1]:
# !pip install mitiq

try:
    import mitiq
    print("Package 'mitiq' is installed.")
except ImportError:
    print("Package 'mitiq' is not installed.")


Package 'mitiq' is installed.


We can grab some other nice things. 

In [2]:
from braket.devices import LocalSimulator
from braket.circuits import Circuit
from noise_models import qd_depol
import json
from qiskit_braket_provider import to_braket, to_qiskit
from mitiq.observable import PauliString, Observable


### Executors

First, we will import the Braket executor. This is essentially a wrapper around the ProgramSet object, and will fail it you try to submit more than the ProgramSet limit for a given device. 


In [3]:
from mitiq_braket_tools import braket_expectation_executor, braket_measurement_executor


The `braket_expectation_estimator` takes in a Braket observable, and then call apply that to the Circuit.

In [4]:
from mitiq.zne import execute_with_zne, RichardsonFactory
from mitiq.executor import Executor
from braket.circuits.observables import Z
from mitiq_braket_tools import _execute_via_program_set

base_circ = Circuit().h(0).cnot(0,1)
base_circ = to_braket(to_qiskit(base_circ, add_measurements=False), basis_gates=["rz","rx", "cx"])


executor = braket_expectation_executor(qd_depol, Z(0) @ Z(1), shots = 10000, verbatim= False, batch_if_possible=False)

print(base_circ)


noisy_result = executor.evaluate(base_circ)[0]
print(f"Noisy Result: {noisy_result}")

factory = RichardsonFactory([1., 3., 5.])
zne_result = execute_with_zne(base_circ, executor, factory=factory)
print(f"Zero-Noise-Extrapolation Result: {zne_result}")

#########

noisy, zne = [], []
for i in range(10):
    noisy.append(executor.evaluate(base_circ)[0])
    zne.append(execute_with_zne(base_circ, executor, factory=factory))

avg_noisy = sum([abs(1-a) for a in noisy])/10
avg_zne = sum([abs(1-a) for a in zne])/10

print(f"{100*(1 - (avg_zne)/(avg_noisy)):.2f}% Average Reduction in Error")

/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/qiskit_braket_provider/providers/adapter.py:903: UserWarning: Device does not support global phase; global phase of 1.5707963267948966 will not be included in Braket circuit
  warnings.warn(


T  : │     0      │     1      │     2      │  3  │
      ┌──────────┐ ┌──────────┐ ┌──────────┐       
q0 : ─┤ Rz(1.57) ├─┤ Rx(1.57) ├─┤ Rz(1.57) ├───●───
      └──────────┘ └──────────┘ └──────────┘   │   
                                             ┌─┴─┐ 
q1 : ────────────────────────────────────────┤ X ├─
                                             └───┘ 
T  : │     0      │     1      │     2      │  3  │
Noisy Result: 0.9486
Zero-Noise-Extrapolation Result: 1.0028249999999992


/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/mitiq/zne/inference.py:88: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(
/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/mitiq/zne/inference.py:88: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(
/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/mitiq/zne/inference.py:88: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(
/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/mitiq/zne/inference.py:88: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(
/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/mitiq/zne/inference.py:88: UserWarning: The input circuit is very short.

83.01% Average Reduction in Error


/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/mitiq/zne/inference.py:88: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(


In comparison, the `braket_measurement_executor` returns a `mitiq.MeasurementResult`, and can support mitiq based observables in the executor. 

In [6]:

zz = Observable(PauliString("ZZ", coeff = 1))

executor_2 = braket_measurement_executor(qd_depol, 10000, False, batch_if_possible=False)


noisy_result = executor_2.evaluate(base_circ, zz)
print(f"Noisy Result: {noisy_result}")

factory = RichardsonFactory([1., 3., 5.])
zne_result = execute_with_zne(base_circ, executor_2, zz,  factory=factory)
print(f"Zero-Noise-Extrapolation Result: {zne_result}")


/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/mitiq/interface/mitiq_braket/conversions.py:116: UserWarning: Measurement gate removed when converting from Cirq to Braket.
  warn("Measurement gate removed when converting from Cirq to Braket.")
/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/mitiq/zne/inference.py:88: UserWarning: The input circuit is very short. This may reduce the accuracy of noise scaling.
  warnings.warn(
/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/mitiq/interface/mitiq_braket/conversions.py:116: UserWarning: Measurement gate removed when converting from Cirq to Braket.
  warn("Measurement gate removed when converting from Cirq to Braket.")
/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/mitiq/interface/mitiq_braket/conversions.py:116: UserWarning: Measurement gate removed when converting from Cirq to Braket.
  warn("Measurement gate removed when conve

Noisy Result: [(0.9472+0j)]
Zero-Noise-Extrapolation Result: (1.0037499999999995+0j)


## Calibrators

We can also use mitiq's built in `Calibrator`, to assess which approaches may be most suitable for a particular circuit. For instance, let's take. 



In [9]:
from mitiq.calibration import Calibrator

cal = Calibrator(executor_2, frontend = "braket")

print(cal.get_cost())


{'noisy_executions': 100, 'ideal_executions': 0}


In [10]:
cal.run()


/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/mitiq/interface/mitiq_braket/conversions.py:116: UserWarning: Measurement gate removed when converting from Cirq to Braket.
  warn("Measurement gate removed when converting from Cirq to Braket.")
/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/mitiq/interface/mitiq_braket/conversions.py:116: UserWarning: Measurement gate removed when converting from Cirq to Braket.
  warn("Measurement gate removed when converting from Cirq to Braket.")
/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/mitiq/interface/mitiq_braket/conversions.py:116: UserWarning: Measurement gate removed when converting from Cirq to Braket.
  warn("Measurement gate removed when converting from Cirq to Braket.")
/Users/sesmart/projects/04_em_mitiq/code/.venv/lib/python3.12/site-packages/mitiq/interface/mitiq_braket/conversions.py:116: UserWarning: Measurement gate removed when converting from 

In [11]:
cal.results.log_results_cartesian()

┌────────────────────────────────────┬─────────────────────────────┬────────────────────────────┬────────────────────────────┬────────────────────────────┐
│ strategy\benchmark                 │ Type: ghz                   │ Type: w                    │ Type: rb                   │ Type: mirror               │
│                                    │ Num qubits: 2               │ Num qubits: 2              │ Num qubits: 2              │ Num qubits: 2              │
│                                    │ Circuit depth: 2            │ Circuit depth: 2           │ Circuit depth: 43          │ Circuit depth: 33          │
│                                    │ Two qubit gate count: 1     │ Two qubit gate count: 2    │ Two qubit gate count: 9    │ Two qubit gate count: 14   │
├────────────────────────────────────┼─────────────────────────────┼────────────────────────────┼────────────────────────────┼────────────────────────────┤
│ Technique: ZNE                     │ ✔                        

Here we can see that mitiq will perform a scan of multiple methods at a lower accuracy, and provide recommendations for which approach achieves the best. With a particular gate structure, folding strategy, etc., and noise rate, this can provide a hands-off approach for assessing viable aproaches.

We can also inspect our executor, looking at relevant outputs and the total number of circuits run. 

In [ ]:
executor_2.calls_to_executor


False

### Summary

In this notebook we saw an overview of mitiq, as well as how to use mitiq with ProgramSets. Generally, error mitigation allows us to reliably expand what is possible on today's quantum computers, and can help deliver more reliable and useful results.  

In subsequent notebooks we will dive into common error mitigation strategies, and provide explicit implementations which you can utilize for systems of interest.

### References


